# makemore: part 3

In [170]:
import pandas as pd
from unidecode import unidecode
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [171]:
def clean(x):
    l = ['-','\'',' ','–']
    for i in l:
        x = x.replace(i,'')
    x = unidecode(x)
    return x.lower()

In [172]:
df = pd.read_csv('ourdata.csv')
df.sample(frac=1,random_state=543)
df.to_csv('ourdata.csv',index=False)

In [173]:
# read in all the words
df.drop_duplicates(inplace=True)
words = df['demonym'][:int(0.8*(df.shape[0]))]
city = df['Place'][:int(0.8*(df.shape[0]))]

In [174]:
words = [clean(word) for word in words]
city = [clean(word) for word in city]

In [175]:
for i in range(len(words)):
    l = len(city[i])-3
    words[i] = words[i][l:]

In [176]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [177]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)

Xtr,  Ytr  = build_dataset(words)

torch.Size([9404, 3]) torch.Size([9404])


In [178]:
class Linear:
  
  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out), generator=g) / fan_in**0.5
    self.bias = torch.zeros(fan_out) if bias else None
  
  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out
  
  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])


class BatchNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      xmean = x.mean(0, keepdim=True) # batch mean
      xvar = x.var(0, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []

n_embd = 15 # the dimensionality of the character embedding vectors
n_hidden = 100 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647) # for reproducibility

C = torch.randn((vocab_size, n_embd),generator=g)
layers = [
  Linear(n_embd * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size),
]
with torch.no_grad():
  layers[-1].gamma *= 0.1
  for layer in layers[:-1]:
    if isinstance(layer, Linear):
      layer.weight *= 1.0 #5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

48659


In [179]:
# same optimization as last time
max_steps = 5000
batch_size = 90
lossi = []
ud = []

for i in range(max_steps):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  
  # forward pass
  emb = C[Xb] # embed the characters into vectors
  x = emb.view(emb.shape[0], -1) # concatenate the vectors
  for layer in layers:
    x = layer(x)
  loss = F.cross_entropy(x, Yb) # loss function
  
  # backward pass
  for layer in layers:
    layer.out.retain_grad() # AFTER_DEBUG: would take out retain_graph
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i<2500 else 0.05 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 100 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())
  with torch.no_grad():
    ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])

  # if i >= 1000:
  #   break # AFTER_DEBUG: would take out obviously to run full optimization

      0/   5000: 3.2830


    100/   5000: 1.9309
    200/   5000: 1.4931
    300/   5000: 1.4624
    400/   5000: 1.6777
    500/   5000: 1.5808
    600/   5000: 1.4797
    700/   5000: 1.4756
    800/   5000: 1.3010
    900/   5000: 1.3279
   1000/   5000: 1.4176
   1100/   5000: 1.6041
   1200/   5000: 1.4830
   1300/   5000: 1.2375
   1400/   5000: 1.4366
   1500/   5000: 1.4516
   1600/   5000: 1.2303
   1700/   5000: 1.6452
   1800/   5000: 1.7355
   1900/   5000: 1.4793
   2000/   5000: 1.4669
   2100/   5000: 1.3942
   2200/   5000: 1.6329
   2300/   5000: 1.3438
   2400/   5000: 1.6331
   2500/   5000: 1.2726
   2600/   5000: 1.5057
   2700/   5000: 1.3351
   2800/   5000: 1.3227
   2900/   5000: 1.6100
   3000/   5000: 0.9892
   3100/   5000: 1.3274
   3200/   5000: 1.3017
   3300/   5000: 1.4195
   3400/   5000: 1.3186
   3500/   5000: 1.4616
   3600/   5000: 1.3999
   3700/   5000: 1.0905
   3800/   5000: 1.2101
   3900/   5000: 1.4203
   4000/   5000: 1.4653
   4100/   5000: 1.2152
   4200/   5000:

In [180]:
n = df.shape[0]
n
Xdev = df[int(0.8*n):int(0.9*n)]['Place']
Ydev = df[int(0.8*n):int(0.9*n)]['demonym']

In [186]:
Xtest = df[int(0.9*n):]['Place']
Ytest = df[int(0.9*n):]['demonym']

In [181]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  x = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  for layer in layers:
    x = layer(x)
  loss = F.cross_entropy(x, y)
  print(split, loss.item())

# put layers into eval mode
for layer in layers:
  layer.training = False
split_loss('train')

train 1.2655551433563232


In [182]:
def pred(city):
  g = torch.Generator().manual_seed(14748347 + 10)
  out = []
  exm = city[-3:]
  context = [stoi[i] for i in exm]
  while True:
    emb = C[torch.tensor([context])]
    x = emb.view(emb.shape[0], -1)
    for layer in layers:
      x = layer(x)
    logits = x
    probs = F.softmax(logits, dim=1)
    ix = torch.argmax(probs).item()
    context = context[1:] + [ix]
    out.append(ix)
    if ix == 0:
      break

  s=(''.join(itos[i] for i in out))
  return (city+s[:-1])

In [190]:
cnt=0
score=0
for x in Xtest:
    #print(cnt)
    y = pred(clean(x))
    #print(y)
    ycorr = clean(Ytest.iloc[cnt])
    #print(ycorr)
    if(ycorr==y):
        score+=1
    cnt+=1
    #print(score/cnt)
print("test accuracy: ",score/cnt)

test accuracy:  0.4387755102040816


In [191]:
import gradio as gr
ui = gr.Interface(fn=pred,inputs="text",outputs="text")
ui.launch()

Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.
